In [1]:
from pathlib import Path
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
api_key  = os.getenv('OPENAI_APIKEY')

In [2]:

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [4]:
vector_store = client.beta.vector_stores.create(name="地震発生マニュアル")

file_paths = ["manual_jishin.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [16]:
print(vector_store.id)

vs_NWNBMQKLxpLRGt3pmkuU786V


In [5]:
assistant = client.beta.assistants.create(
  name="地震マニュアルassistant",
  instructions="You are an expert assistant. Use your knowledge base to answer questions about 東京理科大の地震発生マニュアル.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [10]:

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [11]:
thread = client.beta.threads.create()


In [12]:
prompt = "緊急避難の方法を教えてください"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)


In [14]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)


In [15]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")

    # file_citationがある場合の処理
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

東京理科大学の地震発生時の緊急避難に関する方法は以下の通りです：

1. **屋内にいる場合**:
   - 丈夫な机やテーブルの下に避難し、落下物や家具の転倒から身を守る[0]。
   - 窓や棚の近くにいる場合は、すぐにそこから離れる[1]。
   - エレベータは使用せず、階段を利用して避難する[2]。

2. **火災が発生した場合**:
   - タオルやハンカチで口を覆い、煙から身を守りながら、安全な場所に避難する[2]。
   - 消火できない場合は、ドアや防火戸を閉めて安全な場所へ退避する[4]。

3. **屋外にいる場合**:
   - 門やブロック塀、自動販売機からは速やかに離れる[5]。
   - 公園や空き地など広い場所に避難し、電線に触れないよう注意する[6]。

4. **移動中の場合**:
   - 車や電車にいる場合は、車両から出ないで、乗務員の指示に従うことが推奨されています[7]。

これらの指示は、現場の状況によって適宜修正する必要があります。必ず現場の環境に応じた最適な行動を取るよう心がけてください。
[0] manual_jishin.pdf
[1] manual_jishin.pdf
[2] manual_jishin.pdf
[3] manual_jishin.pdf
[4] manual_jishin.pdf
[5] manual_jishin.pdf
[6] manual_jishin.pdf
[7] manual_jishin.pdf
